### Creating a fake case-control data set for neonatal metabolomics studies 
We often work with case control studies where the goal is to identify factors (metabolites) that may be associated/contribute to a certain medical condition. This is done by comparing patient/subjects who have the condition/disease (the "cases") with patients who do not have the condition/disease but are otherwise similar (the "controls"). 

In studies involving dry-blood spots sampled from newborns (as part of the national newborn screening program), controls are selected to have the same (or very close in time) date of birth as the cases. This is because the metabolimic profiles of newborns show a strong variation with season [***reference***?], possibly reflecting how the mother's  (and the rest of us) diet/ lifetyle varies depedning on the time of year it is. 


#### The code
The code below will create a fake data set having the typical structure of a record listing sample subjects paired into case and control sample types. We create a pandas dataframe to allow nice visaulization of the data and to allow easy export to a csv and/or excel file. 

We create `n_pairs` of case and control pairs, all having a random date of birth within a defined minimum and maximum age. The maximum age difference between each pair can be defined; below we allow a maximum age difference of seven days, but months and years can also be chosen. 

We also define additional sub groups by assigning a random organ from a list of  `n_organ_groups`; this could perhaps be the tissue from were a certain condition is being monitored in the study. A random barcode is assigned to each subject, which could perhaps be an identifer used to identfy the sample in a biobank say. 

For all random data we use the [`Faker` library](https://faker.readthedocs.io/en/master/index.html), with the [faker-biology plugin](https://pypi.org/project/faker-biology/) available for Python. 




### Setup libs

In [2]:
import pandas as pd
import numpy as np

import faker
from faker_biology.physiology import CellType, Organ, Organelle

from dateutil.relativedelta import relativedelta


# init fake object and load plugins
fake = faker.Faker()

# add organ data plugin 
fake.add_provider(Organ)

# or organelle and celtype plugin
# fake.add_provider(Organelle)
# fake.add_provider(CellType)

### Set parameters

In [3]:
# save
save_to_excel = False

# number of case control pairs in fake study 
n_pairs = 523 

# age of subjects
min_age = 15 # years
max_age = 40 # years

# max age difference between case and control
max_diff_years = 0
max_diff_months = 0
max_diff_days = 7

# number of sub groups; here organs to simulate some attribute of the disease/condition
n_groups = 5
organs = [fake.organ() for _ in range(0,n_groups)]

variables = ["pair_ID", "specimen_ID", "object", "date_of_birth", "barcode", "organ"]

# create dict to hold the fake data 
fake_data = {}
for v in variables:
    fake_data.setdefault(v, [])

### Create fake data

In [4]:
for pair_id in range(0, n_pairs):
    
    # Choose a random sub group
    case_control_organ = np.random.choice(organs)
    
    # --- Controls ---
    dob_control = fake.date_of_birth(minimum_age=min_age, maximum_age=max_age)
    spec_id_control = np.random.randint(1,n_pairs*2+1)
    
    fake_data["pair_ID"].append(pair_id)
    fake_data["object"].append("Control")
    fake_data["specimen_ID"].append(spec_id_control)
    fake_data["date_of_birth"].append(dob_control)
    fake_data["barcode"].append(fake.ean8())
    fake_data["organ"].append(case_control_organ)
    
    # --- Cases ---
    
    # let the case be born within relativedelta time from control
    dob_case = fake.date_between_dates(dob_control, 
                            dob_control + relativedelta(years=max_diff_years, 
                                          month=max_diff_months, 
                                          days=max_diff_days)
                            )
    
    # let case and control specimen numbers be contiguous
    spec_id_case = spec_id_control + 1
    
    fake_data["pair_ID"].append(pair_id)
    fake_data["object"].append("Case")
    fake_data["specimen_ID"].append(spec_id_case)
    fake_data["date_of_birth"].append(dob_case)
    fake_data["barcode"].append(fake.ean8())
    fake_data["organ"].append(case_control_organ)

In [5]:
fake_data_df = pd.DataFrame(fake_data)

if save_to_excel:
    filename = f"fake_case_control_Npairs_{n_pairs}_Ngroups_{n_groups}.xlsx"
    fake_data_df.to_excel("output/" + filename)

In [8]:
fake_data_df

,pair_ID,specimen_ID,object,date_of_birth,barcode,organ
0,0,986,Control,1986-05-23,29724024,Joints
1,0,987,Case,1986-05-26,25217650,Joints
2,1,677,Control,1989-08-28,63963250,Joints
3,1,678,Case,1989-09-02,45443497,Joints
4,2,122,Control,1986-04-04,33163055,Olfactory epithelium
...,...,...,...,...,...,...
1041,520,35,Case,1984-04-18,15278739,Joints
1042,521,632,Control,1994-07-07,81891306,Olfactory epithelium
1043,521,633,Case,1994-07-08,97756927,Olfactory epithelium
1044,522,798,Control,1989-05-18,59066996,Olfactory epithelium
